# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.tail()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
537,537,Kaupanger,100,NO,1578004259,97,61.18,7.24,32.34,17.65
538,538,Kimbe,0,PG,1578004259,58,-5.56,150.15,89.87,1.83
539,539,Vila,86,IT,1578004259,74,45.18,8.40,41.00,1.12
540,540,Novikovo,99,RU,1578004260,96,51.15,37.89,29.34,7.90
541,541,Porbandar,45,IN,1578004260,67,21.64,69.61,69.42,17.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#https://people.revoledu.com/kardi/tutorial/Python/Displaying+Locations+using+Heatmap.html
gmaps.configure(api_key=g_key)
latLong_df = cities_df[["Lat","Lng"]]
humidity_s = cities_df["Humidity"].astype(float)

def drawHeatMap(location, val, zoom, intensity, radius):
    # setting the data and parameters
    heatmap_layer = gmaps.heatmap_layer(location, val, dissipating = True)
    heatmap_layer.max_intensity = intensity
    heatmap_layer.point_radius = radius
    # draw the heatmap into a figure
    fig = gmaps.figure()
    fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
    fig.add_layer(heatmap_layer)
    return fig

zoom=10
intensity=100
radius=5

# call the function to draw the heatmap
fig = drawHeatMap(latLong_df, humidity_s, zoom, intensity, radius)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
possibleVacaSpots_DF = cities_df.dropna()
possibleVacaSpots_DF = possibleVacaSpots_DF[(possibleVacaSpots_DF['Max Temperature'] >= 68) & (possibleVacaSpots_DF['Max Temperature'] <= 75)]
possibleVacaSpots_DF = possibleVacaSpots_DF[possibleVacaSpots_DF['Wind Speed'] <= 15]

newHumidity_s=possibleVacaSpots_DF["Humidity"].astype(float)
radius=7
fig = drawHeatMap(possibleVacaSpots_DF[["Lat","Lng"]], newHumidity_s, zoom, intensity, radius)
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = possibleVacaSpots_DF
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.drop(columns='Unnamed: 0')
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
0,Dingle,100,PH,1578004098,94,11.00,122.67,73.13,9.53,
1,Werota,73,ET,1578004101,62,11.92,37.70,72.72,9.82,
2,Anantapur,0,IN,1578004012,82,14.68,77.61,72.90,5.77,
3,Seidu,0,NG,1578004110,32,8.32,4.47,69.80,11.41,
4,Airai,100,TL,1578004112,92,-8.93,125.41,68.20,1.43,


In [55]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#added a sort by review descending with strong bias, cause who wants to stay at the first hotel they see god bless motel 6 but I would never stay there....
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key,
    "sort":"review-rating:d:s"
}

for index, row in hotel_df.iterrows():
    # Build query URL
    params["location"] = "{},{}".format(row["Lat"], row["Lng"])
    hotels = requests.get(base_url, params=params).json()
    #print(hotels.url)
    #print(json.dumps(hotels, indent=4, sort_keys=True))
    #print(hotels["status"])
    #print(index)
    if hotels["status"] == "OK":
        #print("Found")
        #print(hotels['results'][0]['name'])
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    else:
        hotel_df.loc[index, "Hotel Name"] = ""
    
hotel_df.head()    

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
0,Dingle,100,PH,1578004098,94,11.00,122.67,73.13,9.53,
1,Werota,73,ET,1578004101,62,11.92,37.70,72.72,9.82,"JACARANDA HOTEL ,BAHIR DAR"
2,Anantapur,0,IN,1578004012,82,14.68,77.61,72.90,5.77,Hotel Rajahamsa
3,Seidu,0,NG,1578004110,32,8.32,4.47,69.80,11.41,
4,Airai,100,TL,1578004112,92,-8.93,125.41,68.20,1.43,


In [56]:

hotel_df.tail()    

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
56,Nome,90,US,1578004253,94,30.04,-94.42,69.80,5.82,
57,Mbigou,100,GA,1578004254,98,-1.90,11.91,68.16,3.29,
58,Huarmey,58,PE,1578004254,77,-10.07,-78.15,68.77,5.08,Hotel Miramar
59,Morondava,94,MG,1578004256,94,-20.30,44.28,74.35,3.20,Palissandre Cote Ouest resort & SPA
60,Lompoc,1,US,1578004256,58,34.64,-120.46,71.60,6.93,Embassy Suites by Hilton Lompoc Central Coast


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [61]:
# Add marker layer ontop of heat map
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))